In [1]:
import tensorflow as tf

In [2]:
# Model parameters
W = tf.Variable([.3], dtype=tf.float32, name ='W')
b = tf.Variable([-.3], dtype=tf.float32, name = 'b')
# Model input and output
x = tf.placeholder(tf.float32, name = 'input')
output = tf.add(tf.multiply(W,x),b, name='output')
y = tf.placeholder(tf.float32)

In [3]:
# loss
loss = tf.reduce_sum(tf.square(W*x+b - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [4]:
saver = tf.train.Saver()
init_op = tf.global_variables_initializer()
sess = tf.Session()

sess.run(init_op)
# save the graph
tf.train.write_graph(sess.graph_def, '.','tensor.pbtxt')  

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})
    
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


In [5]:
#assign W and b
sess.run(tf.assign(W, curr_W))
sess.run(tf.assign(b, curr_b))

#save a checkpoint file, which will store the above assignment  
saver.save(sess, './tensor.ckpt')

'./tensor.ckpt'

In [6]:
MODEL_NAME = 'tensor'

# Freeze the graph

input_graph_path = MODEL_NAME+'.pbtxt'
checkpoint_path = './'+MODEL_NAME+'.ckpt'
input_saver_def_path = ""
input_binary = False
output_node_names = "output"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = 'frozen_'+MODEL_NAME+'.pb'
output_optimized_graph_name = 'optimized_'+MODEL_NAME+'.pb'
clear_devices = True

# library for freezing the graph
from tensorflow.python.tools import freeze_graph

freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_frozen_graph_name, clear_devices, "")

INFO:tensorflow:Restoring parameters from ./tensor.ckpt
INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.
7 ops in the final graph.


In [8]:
# library for optmising inference
from tensorflow.python.tools import optimize_for_inference_lib

input_graph_def = tf.GraphDef()
with tf.gfile.Open(output_frozen_graph_name, "rb") as f:
    data = f.read()
    input_graph_def.ParseFromString(data)

output_graph_def = optimize_for_inference_lib.optimize_for_inference(
        input_graph_def,
        ["input"], # an array of the input node(s)
        ["output"], # an array of output nodes
        tf.float32.as_datatype_enum)

# Save the optimized graph

f = tf.gfile.FastGFile(output_optimized_graph_name, "w")
f.write(output_graph_def.SerializeToString())